# Estudo de Caso 2 - Processo de Inicialização e Otimização

Este Jupyter Notebook tem como objetivo a demonstração prática dos efeitos dos diferentes processos de inicialização e otimização para modelos com redes neurais. 

O processo de otimização em Deep Learning é o processo de encontrar os melhores pesos para a rede neural, de modo que ela possa realizar bem em tarefas específicas, como classificação ou regressão. Alguns exemplos são o SGD, Momentum e Adagrad.

O processo de inicialização em Deep Learning é o processo de definir os valores iniciais dos pesos das camadas da rede neural antes do treinamento começar. Alguns exemplos são inicialização aleatória, inicialização com zero e com pequenos valores próximo a zero.

## 1. Carga e instalação dos pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print('A versão da linguagem python neste Jupyter Notebook é: ', python_version())

A versão da linguagem python neste Jupyter Notebook é:  3.9.13


In [2]:
# Instalação versão do torch
#!pip install -q torch==1.13.0

In [3]:
# Instalação versão do Pytorch
#!pip install -q torchvision==0.14.0

In [4]:
# Carga dos pacotes/funções

# Pacote para trabalhar no SO
import os

# Pacotes de visualização e Processamento de dados
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import shutil

# Pacote Torch
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

# Pacote Torchvison
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchvision import models

# Relatório do ambiente de desenvolvimento
import gc
import types
import pkg_resources
import pytorch_lightning as pl
%matplotlib inline

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Estudo de Caso 2 - Processos de Otimização/Inicialização" --iversions

Author: Estudo de Caso 2 - Processos de Otimização/Inicialização

torch            : 1.13.0
numpy            : 1.21.5
matplotlib       : 3.5.2
torchvision      : 0.14.0
pytorch_lightning: 1.8.3



## 2. Preparação dos Dados

### 2.1 Criação das pastas no diretório

In [6]:
pasta_dados_originais = 'data'

In [7]:
nomes_classes = ['cloudy', 'desert', 'green_area', 'water']

In [8]:
# Loop para criar as pastas
for i in range(len(nomes_classes)):
    
    # Extrai o nome de cada classe
    class1 = '/' + nomes_classes[i]
    
    # Cria as pastas
    os.makedirs('dados/treino/' + nomes_classes[i])
    os.makedirs('dados/val/' + nomes_classes[i])
    os.makedirs('dados/teste/' + nomes_classes[i])

### 2.2 Input das imagens nas pastas

In [9]:
# Loop
for k in range(len(nomes_classes)):
    
    # Extrai um nome de classe
    nome_classe = nomes_classes[k]
    
    # Define a fonte
    src = pasta_dados_originais + "/" + nome_classe 
    
    # Mostra qual classe estamos processando
    print("\nClasse:", nomes_classes[k])
    
    # Lista o conteúdo da pasta
    allFileNames = os.listdir(src)
    
    # "Embaralha" os dados
    np.random.shuffle(allFileNames)
    
    # Divisão = 70% treino, 15% teste, 15% validação 
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)*0.7), int(len(allFileNames)*0.85)])


    # Nome dos arquivos
    train_FileNames = [src + '/' + name for name in train_FileNames.tolist()]
    val_FileNames = [src + '/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src + '/' + name for name in test_FileNames.tolist()]

    # Print
    print('Número Total de Imagens: ', len(allFileNames))
    print('Imagens de Treino: ', len(train_FileNames))
    print('Imagens de Validação: ', len(val_FileNames))
    print('Imagens de Teste: ', len(test_FileNames))
    
    # Copia as imagens
    for name in train_FileNames:
        shutil.copy(name, "dados/treino/" + nome_classe)
    
    for name in val_FileNames:
        shutil.copy(name, "dados/val/" + nome_classe)
        
    for name in test_FileNames:
        shutil.copy(name, "dados/teste/" + nome_classe)  


Classe: cloudy
Número Total de Imagens:  1500
Imagens de Treino:  1050
Imagens de Validação:  225
Imagens de Teste:  225

Classe: desert
Número Total de Imagens:  1131
Imagens de Treino:  791
Imagens de Validação:  170
Imagens de Teste:  170

Classe: green_area
Número Total de Imagens:  1500
Imagens de Treino:  1050
Imagens de Validação:  225
Imagens de Teste:  225

Classe: water
Número Total de Imagens:  1500
Imagens de Treino:  1050
Imagens de Validação:  225
Imagens de Teste:  225


### 2.3 Verificando o ambiente de desenvolvimento

In [10]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('matplotlib', '3.5.2'), ('numpy', '1.21.5'), ('torch', '1.13.0'), ('torchvision', '0.14.0')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  1.13.0+cpu
Versão do Lightning:  1.8.3

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------


Limpando a Memória da GPU (se disponível):  None

------------------------------------------Fim da Checagem-------------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


### 2.4 Funções auxiliares para input dos dados no dispositivo de treino

In [11]:
# Função para obter o device
# Se disponível usamos GPU, caso contrário usamos CPU.

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [12]:
# Função para enviar um tensor para o device
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking = True)

In [13]:
# Classe para enviar os dataloaders para o device
class DeviceDataLoader():
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

In [14]:
# Visualiza o device
device = get_default_device()
device

device(type='cpu')

## 3. Pré-Processamento dos Dados

### 3.1 Transformação dos dados

In [15]:
# Processo de normalização dos dados (médias e desvio padrão para cada canal de cor)
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

In [16]:
# Define as transformações dos dados
transforms = transforms.Compose([transforms.RandomCrop(64, padding_mode = 'reflect'),
                                 transforms.Resize(64),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(*stats, inplace = True)])

### 3.2 Carga dos dados

In [17]:
# Dados de treino
dados_treino = datasets.ImageFolder('dados/treino', transform = transforms)

In [18]:
# Dados de validação
dados_valid = datasets.ImageFolder('dados/val', transform = transforms)

In [19]:
# Dados de teste
dados_teste = datasets.ImageFolder('dados/teste', transform = transforms)

In [20]:
# Visualiza os dados
print(f"Número Total de Imagens de Treino: {len(dados_treino)}")
print(f"Número Total de Imagens de Validação: {len(dados_valid)}")
print(f"Número Total de Imagens de Teste: {len(dados_teste)}") 

Número Total de Imagens de Treino: 3941
Número Total de Imagens de Validação: 845
Número Total de Imagens de Teste: 845


### 3.3 Prepara os dataloaders

In [21]:
# Parâmetros para os dataloaders
batch_size = 8
shuffle = True
num_workers = 2
pin_memory = True

In [22]:
# Data loader de treino
data_loader_treino = DataLoader(dados_treino, 
                                batch_size, 
                                shuffle = shuffle, 
                                num_workers = num_workers, 
                                pin_memory = pin_memory)

In [23]:
# Data loader de validação
data_loader_valid = DataLoader(dados_valid, 
                               batch_size, 
                               num_workers = num_workers, 
                               pin_memory = pin_memory)

In [24]:
# Data loader de teste
data_loader_teste = DataLoader(dados_teste, 
                               batch_size * 2, 
                               num_workers = num_workers, 
                               pin_memory = pin_memory)

In [25]:
# Função para "desnormalizar" os dados
def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

## 4. Arquitetura do Modelo

In [26]:
# Função para calcular a acurácia
def accuracy(outputs, labels):
    
    # Extrai o maior valor dos outputs (classe com maior probabilidade)
    _, preds = torch.max(outputs, dim = 1)
    
    # Retorna a acurácia comparando as previsões com os valores reais
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [27]:
# Classificador Base
class ImageClassificationBase(nn.Module):
    
    # Método construtor
    def __init__(self):
        super().__init__()
        self.reset_parameters()
        
    # Método de resert dos pesos (parâmetros)
    def reset_parameters(self):
        self.weight = torch.empty(3, 4)
        nn.init.kaiming_uniform_(self.weight, mode = 'fan_in', nonlinearity = 'relu')
    
    # Método para o passo de treino
    def training_step(self, batch):
        
        # Recebe um batch
        images, labels = batch 
        
        # Faz uma previsão
        out = self(images)    
        
        # Calcula o erro
        loss = F.cross_entropy(out, labels) 
        
        return loss
    
    # Método para o passo de validação
    def validation_step(self, batch):
        
        # Recebe um batch
        images, labels = batch 
        
        # Faz uma previsão
        out = self(images)    
        
        # Calcula o erro
        loss = F.cross_entropy(out, labels)   
        
        # Calcula a acurácia
        acc = accuracy(out, labels)     
        
        return {'erro_valid': loss.detach(), 'acc_valid': acc}
        
    # Método para o passo de validação ao final de uma época
    def validation_epoch_end(self, outputs):
        batch_losses = [x['erro_valid'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['acc_valid'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()     
        return {'erro_valid': epoch_loss.item(), 'acc_valid': epoch_acc.item()}
    
    # Método para o fim de cada época
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, erro_treino: {:.4f}, erro_valid: {:.4f}, acc_valid: {:.4f}".format(
            epoch, result['lrs'][-1], result['erro_treino'], result['erro_valid'], result['acc_valid']))

In [28]:
# Bloco de convolução
def conv_block(in_channels, out_channels, pool = False):
    
    # Camadas
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace = True)]
    
    # Se pool = True, adiciona a camada de MaxPooling
    if pool: 
        layers.append(nn.MaxPool2d(2))
        
    return nn.Sequential(*layers)

In [29]:
# Arquitetura ResNet
class ResNet(ImageClassificationBase):
    
    # Método construtor
    def __init__(self, in_channels, num_classes):
        
        # Inicializa o construtor da classe mãe
        super().__init__()
        
        # Conv1
        self.conv1 = conv_block(in_channels, 64)
        
        # Conv2
        self.conv2 = conv_block(64, 128, pool = True)
        
        # Sequência de convs
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
        # Conv3
        self.conv3 = conv_block(128, 256, pool = True)
        
        # Conv4
        self.conv4 = conv_block(256, 512, pool = True)
        
         # Sequência de convs
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        # Camada de classificação
        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
    
    # Método forward
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [30]:
# Cria o objeto do modelo considerando 3 canais de cores nas imagens e 4 classes de saída
modelo = to_device(ResNet(3, 4), device)

# Visualiza o modelo
modelo

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tru

## 5. Carga dos Dados no Device

In [31]:
# Dados de treino
data_loader_treino = DeviceDataLoader(data_loader_treino, device)

# Dados de validação
data_loader_valid = DeviceDataLoader(data_loader_valid, device)

# Dados de teste
data_loader_teste = DeviceDataLoader(data_loader_teste, device)

## 6. Funções para Loop de Treinamento

In [32]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [33]:
# Função para obter a taxa de aprendizado
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [34]:
# Função de treino
def treina_modelo(epochs, 
                  max_lr, 
                  model, 
                  train_loader, 
                  val_loader, 
                  weight_decay = 0, 
                  grad_clip = None, 
                  opt_func = torch.optim.SGD):
    
    # Limpa o cache da GPU
    torch.cuda.empty_cache()
    
    # Lista para o histórico de treino
    history = []
    
    # Define o otimizador com a função opt_func
    if opt_func == torch.optim.Adam:
        optimizer = opt_func(model.parameters(), max_lr, weight_decay = weight_decay)
    elif opt_func == torch.optim.SGD:
        optimizer = opt_func(model.parameters(), max_lr, weight_decay = weight_decay, momentum = 0.9)
    elif opt_func == torch.optim.RMSprop:
        optimizer = opt_func(model.parameters(), max_lr, weight_decay = weight_decay, eps = 1e-9)
    
    # Configura o agendador de taxa de aprendizado de um ciclo
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                max_lr, 
                                                epochs = epochs, 
                                                steps_per_epoch = len(train_loader))
    
    # Loop de treino
    for epoch in range(epochs):
        
        # Coloca o modelo em modo de treino
        model.train()
        
        # Listas de controle
        train_losses = []
        lrs = []
        
        # Loop para extrair os batches
        for batch in train_loader:
            
            # Passo de treino
            loss = model.training_step(batch)
            
            # Armazena o erro
            train_losses.append(loss)
            
            # Backpropagation para calcular os gradientes
            loss.backward()
            
            # Gradient clipping 
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            # Passo de otimização
            optimizer.step()
            
            # Zera os gradientes
            optimizer.zero_grad()
            
            # Registra e atualiza a taxa de aprendizado
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validação
        result = evaluate(model, val_loader)
        result['erro_treino'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
        
    return history

### 5.1 Definições de Hiperparâmetros

In [35]:
# Definição dos hiperparâmetros
epochs = 5
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4

### 5.2 Otimizadores

#### 5.2.1 ADAM

In [36]:
# Função de otimização
opt_func = torch.optim.Adam

In [37]:
# Treinamento
history = treina_modelo(epochs, 
                        max_lr, 
                        modelo, 
                        data_loader_treino, 
                        data_loader_valid, 
                        grad_clip = grad_clip, 
                        weight_decay = weight_decay, 
                        opt_func = opt_func)

Epoch [0], last_lr: 0.00759, erro_treino: 0.9517, erro_valid: 1.3179, acc_valid: 0.7134
Epoch [1], last_lr: 0.00950, erro_treino: 1.2679, erro_valid: 0.3811, acc_valid: 0.8375
Epoch [2], last_lr: 0.00611, erro_treino: 0.7223, erro_valid: 0.4184, acc_valid: 0.8243
Epoch [3], last_lr: 0.00188, erro_treino: 0.5394, erro_valid: 0.2738, acc_valid: 0.8943
Epoch [4], last_lr: 0.00000, erro_treino: 0.4418, erro_valid: 0.2268, acc_valid: 0.9167


In [38]:
# Avaliação com dados de teste
aval = [evaluate(modelo, data_loader_teste)]
print(aval)

[{'erro_valid': 0.24813513457775116, 'acc_valid': 0.9171807169914246}]


#### 5.2.2 SGD

In [39]:
# Função de otimização
opt_func = torch.optim.SGD

In [40]:
# Treinamento
history = treina_modelo(epochs, 
                        max_lr, 
                        modelo, 
                        data_loader_treino, 
                        data_loader_valid, 
                        grad_clip = grad_clip, 
                        weight_decay = weight_decay, 
                        opt_func = opt_func)

Epoch [0], last_lr: 0.00759, erro_treino: 0.4452, erro_valid: 0.2574, acc_valid: 0.9120
Epoch [1], last_lr: 0.00950, erro_treino: 0.4394, erro_valid: 0.2650, acc_valid: 0.8856
Epoch [2], last_lr: 0.00611, erro_treino: 0.4254, erro_valid: 0.3389, acc_valid: 0.8642
Epoch [3], last_lr: 0.00188, erro_treino: 0.4376, erro_valid: 0.2360, acc_valid: 0.9050
Epoch [4], last_lr: 0.00000, erro_treino: 0.4037, erro_valid: 0.2392, acc_valid: 0.8967


In [41]:
# Avaliação com dados de teste
aval = [evaluate(modelo, data_loader_teste)]
print(aval)

[{'erro_valid': 0.27395811676979065, 'acc_valid': 0.8768141865730286}]


#### 5.2.3 RMSprop

In [42]:
# Função de otimização
opt_func = torch.optim.RMSprop

In [ ]:
# Treinamento
history = treina_modelo(epochs, 
                        max_lr, 
                        modelo, 
                        data_loader_treino, 
                        data_loader_valid, 
                        grad_clip = grad_clip, 
                        weight_decay = weight_decay, 
                        opt_func = opt_func)

Epoch [0], last_lr: 0.00759, erro_treino: 2.0377, erro_valid: 24.0316, acc_valid: 0.4224
Epoch [1], last_lr: 0.00950, erro_treino: 5.4999, erro_valid: 2.4756, acc_valid: 0.4524
Epoch [2], last_lr: 0.00611, erro_treino: 6.3445, erro_valid: 4.1171, acc_valid: 0.6191


In [ ]:
# Avaliação com dados de teste
aval = [evaluate(modelo, data_loader_teste)]
print(aval)

## 6. Processos de Inicialização

### 6.1 Inicialização Glorot (Xavier)

In [ ]:
# Classificador Base
class ImageClassificationBase(nn.Module):
    
    # Método construtor
    def __init__(self):
        super().__init__()
        self.reset_parameters()
        
    # Método de resert dos pesos (parâmetros)
    def reset_parameters(self):
        self.weight = torch.empty(3, 4)
        nn.init.xavier_normal_(self.weight)
    
    # Método para o passo de treino
    def training_step(self, batch):
        
        # Recebe um batch
        images, labels = batch 
        
        # Faz uma previsão
        out = self(images)    
        
        # Calcula o erro
        loss = F.cross_entropy(out, labels) 
        
        return loss
    
    # Método para o passo de validação
    def validation_step(self, batch):
        
        # Recebe um batch
        images, labels = batch 
        
        # Faz uma previsão
        out = self(images)    
        
        # Calcula o erro
        loss = F.cross_entropy(out, labels)   
        
        # Calcula a acurácia
        acc = accuracy(out, labels)     
        
        return {'erro_valid': loss.detach(), 'acc_valid': acc}
        
    # Método para o passo de validação ao final de uma época
    def validation_epoch_end(self, outputs):
        batch_losses = [x['erro_valid'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['acc_valid'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()     
        return {'erro_valid': epoch_loss.item(), 'acc_valid': epoch_acc.item()}
    
    # Método para o fim de cada época
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, erro_treino: {:.4f}, erro_valid: {:.4f}, acc_valid: {:.4f}".format(
            epoch, result['lrs'][-1], result['erro_treino'], result['erro_valid'], result['acc_valid']))

In [ ]:
# Cria o objeto do modelo considerando 3 canais de cores nas imagens e 4 classes de saída
modelo = to_device(ResNet(3, 4), device)

In [ ]:
# Função de otimização
opt_func = torch.optim.SGD

In [ ]:
# Treinamento
history = treina_modelo(epochs, 
                        max_lr, 
                        modelo, 
                        data_loader_treino, 
                        data_loader_valid, 
                        grad_clip = grad_clip, 
                        weight_decay = weight_decay, 
                        opt_func = opt_func)

In [ ]:
# Avaliação com dados de teste
aval = [evaluate(modelo, data_loader_teste)]
print(aval)

### 6.3 Inicialização dos Pesos com Zero

In [ ]:
# Classificador Base
class ImageClassificationBase(nn.Module):
    
    # Método construtor
    def __init__(self):
        super().__init__()
        self.reset_parameters()
        
    # Método de resert dos pesos (parâmetros)
    def reset_parameters(self):
        self.weight = torch.empty(3, 4)
        nn.init.zeros_(self.weight)
    
    # Método para o passo de treino
    def training_step(self, batch):
        
        # Recebe um batch
        images, labels = batch 
        
        # Faz uma previsão
        out = self(images)    
        
        # Calcula o erro
        loss = F.cross_entropy(out, labels) 
        
        return loss
    
    # Método para o passo de validação
    def validation_step(self, batch):
        
        # Recebe um batch
        images, labels = batch 
        
        # Faz uma previsão
        out = self(images)    
        
        # Calcula o erro
        loss = F.cross_entropy(out, labels)   
        
        # Calcula a acurácia
        acc = accuracy(out, labels)     
        
        return {'erro_valid': loss.detach(), 'acc_valid': acc}
        
    # Método para o passo de validação ao final de uma época
    def validation_epoch_end(self, outputs):
        batch_losses = [x['erro_valid'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['acc_valid'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()     
        return {'erro_valid': epoch_loss.item(), 'acc_valid': epoch_acc.item()}
    
    # Método para o fim de cada época
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, erro_treino: {:.4f}, erro_valid: {:.4f}, acc_valid: {:.4f}".format(
            epoch, result['lrs'][-1], result['erro_treino'], result['erro_valid'], result['acc_valid']))

In [ ]:
# Cria o objeto do modelo considerando 3 canais de cores nas imagens e 4 classes de saída
modelo = to_device(ResNet(3, 4), device)

In [ ]:
# Função de otimização
opt_func = torch.optim.SGD

In [ ]:
# Treinamento
history = treina_modelo(epochs, 
                        max_lr, 
                        modelo, 
                        data_loader_treino, 
                        data_loader_valid, 
                        grad_clip = grad_clip, 
                        weight_decay = weight_decay, 
                        opt_func = opt_func)

In [ ]:
# Avaliação com dados de teste
aval = [evaluate(modelo, data_loader_teste)]
print(aval)